In [1]:
# This script does

# 1. add x y information to deduplicated rows /
# 2. replace lat lon with x y values for good matches /
# 3. remove columns added during matching step

import pandas as pd
import numpy as np

import mitosheet

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
df = pd.read_csv('output/matched_a.csv', low_memory=False)

In [69]:
# for testing, only keep cols we're interested in
# df = df[['localfile', 'business_name', 'latitude', 'longitude', 'street_no', 'street_name', 'city', 'province', 'geo_source', 'formatted_en', 'matches_r', 'ratio', 'x', 'y', 'csdname_oda', 'keep_match']]


In [8]:
df_input = df

In [77]:
df = df_input

In [68]:
df = df.head(5000)

#### Add x and y for deduplicated rows

In [78]:
# sort dataframe by deduplicated variables, so that deduplicated rows appear together
# this way we can check for matches against previous rows
df.sort_values(['province', 'city', 'street_no', 'formatted_en'], inplace=True)
# df.reset_index(inplace=True)

In [79]:
# add in missing matching values for previously de-duplicated rows

for i in range(len(df)):
    
    # check if the street number is the same as the previous row
    if (i > 1):
        if (df.at[i , 'street_no'] == df.at[i - 1, 'street_no']):
            # then street name
            if (df.at[i, 'formatted_en'] == df.at[i - 1, 'formatted_en']):
                # then city
                if (df.at[i, 'city'] == df.at[i - 1, 'city']):
                    # then province
                    if (df.at[i, 'province'] == df.at[i - 1, 'province']):
                        # we might also want to check the source is the same
                        
                        # if they are all the same, then take the value of x
                        # if x is not na or 0, assign it to the previous row
                        
                        # check if previous value had an 'x' 
                        x = df.at[i - 1, 'x']
                        if (not pd.isna(x) and x != 0):
                            # if yes, then assign it to the current row
                            df.at[i, 'x'] = x
                            
                            y = df.at[i - 1, 'y']
                            if (not pd.isna(y) and y != 0):
                                df.at[i, 'y'] = y
                    
                            # also add the other columns from the matching stage
                            df.at[i, 'matches_r'] = df.at[i - 1, 'matches_r']
                            df.at[i, 'ratio'] = df.at[i - 1, 'ratio']
                            df.at[i, 'csdname_oda'] = df.at[i - 1, 'csdname_oda']
                            df.at[i, 'keep_match'] = df.at[i - 1, 'keep_match']



### Remove lat lons for non-matches

In [80]:
# we use two columns to decide whether to keep a match or not
# if keep_match is yes AND ratio > 92 AND no existing lat/lon, then we use xy values for lat/lon
# we also need to say that we used the matched geo source: oda_match

# df['new'] = np.where((df['keep_match'] == 'yes') & (df['ratio'] > 92) & (df['longitude'].isna()), df['x'], '')
df['geo_source'] = np.where((df['keep_match'] == 'yes') & (df['ratio'] > 92) & (df['longitude'].isna()), 'oda_match', df['geo_source'])
df['latitude'] = np.where((df['keep_match'] == 'yes') & (df['ratio'] > 92) & (df['latitude'].isna()), df['y'], df['latitude'])
df['longitude'] = np.where((df['keep_match'] == 'yes') & (df['ratio'] > 92) & (df['longitude'].isna()), df['x'], df['longitude'])


In [73]:
df.at[300,'new']

'-115.57202'

In [6]:
# finally, we clean up columns that are not being used
df = df.drop(columns=['x', 'y', 'ratio', 'matches_r'])


In [81]:
# df.to_csv('output/matched.csv', index=False)
df.to_csv('output/matched.csv', index=False)
# we expect for x and y to be assigned to our repeats

In [10]:
mitosheet.sheet(df, analysis_to_replay="id-vahppewxqg")

MitoWidget(analysis_data_json='{"analysisName": "id-vahppewxqg", "analysisToReplay": null, "code": [], "stepSu…

In [ ]:
from mitosheet import *; register_analysis("id-zmthoxkzcq");
    
# Imported matched.csv
import pandas as pd
matched = pd.read_csv(r'/home/jovyan/ODBiz/5a-Matching/output/matched.csv')
